In [21]:
import numpy as np
import pandas as pd
import seaborn as sb
% matplotlib inline
from datetime import datetime, timedelta

from bs4 import BeautifulSoup as bs
import requests
import re

In [50]:
# Scrape data

years = range(2002, 2018)
letters = [chr(x + ord('A')) for x in range(0, 26)]

URL = "https://services.datasport.com/{}/lauf/lamara/ALFAA.HTM"

rankings = {}
headers = {}
for year in years:
    rankings[str(year)] = []
    for letter in letters:
        r = requests.get(URL.format(year))
        soup = bs(r.text, 'html.parser')
        title = soup.find('title').getText()
        if(title != 'Adresse nicht vorhanden / The address is not available'):
            data = soup.findAll('font', {'size':'2'})

            print(year)
            headers[str(year)] = data[0].getText()
            rankings[str(year)].extend(re.compile('¦ *\r\n').split(data[1].getText()))


2002
2002
2002
2002
2002
2002
2002
2002
2002
2002
2002
2002
2002
2002
2002
2002
2002
2002
2002
2002
2002
2002
2002
2002
2002
2002
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2003
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2004
2005
2005
2005
2005
2005
2005
2005
2005
2005
2005
2005
2005
2005
2005
2005
2005
2005
2005
2005
2005
2005
2005
2005
2005
2005
2005
2006
2006
2006
2006
2006
2006
2006
2006
2006
2006
2006
2006
2006
2006
2006
2006
2006
2006
2006
2006
2006
2006
2006
2006
2006
2006
2007
2007
2007
2007
2007
2007
2007
2007
2007
2007
2007
2007
2007
2007
2007
2007
2007
2007
2007
2007
2007
2007
2007
2007
2007
2007
2008
2008
2008
2008
2008
2008
2008
2008
2008
2008
2008
2008
2008
2008
2008
2008
2008
2008
2008
2008
2008
2008
2008
2008
2008
2008
2009
2009
2009
2009
2009
2009
2009
2009
2009
2009
2009
2009
2009
2009
2009
2009
2009
2009


In [53]:
def parse_columns(ranking, header):
    column_structure = {}
    column_mask = np.ones(np.array(list(ranking[0])).shape, dtype=bool)
    spaces_mask = np.ones(np.array(list(ranking[0])).shape)*32
    
    # create mask of space positions for field delimiter detection
    for line in ranking[:-1]:
        cur_line = np.array(list(line)).view(np.uint32)
        
        try:
            column_mask = np.logical_and(column_mask, np.equal(cur_line, spaces_mask))
        except:
            print("Skipped line:\n", line)
            print(cur_line.shape, column_mask.shape)
            pass
        
    # include header in space position mask
    header_int = np.array(list(header[:column_mask.shape[0]])).view(np.uint32)
    column_mask = np.logical_and(column_mask, np.equal(header_int, spaces_mask))
        
    # find field delimiter positions
    inside_space_col = False
    delimiters = []
    for i, is_space_col in enumerate(list(column_mask)):
        if is_space_col:
            if not inside_space_col:
                inside_space_col = True
                delimiters.append(i)
                
        elif inside_space_col:
            inside_space_col = False
            
    return delimiters
    

def separate_cols(rankings, headers):
    dfs = {}
    for year in rankings:
        ranking = rankings[year]
        header = headers[year]
        delimiters = parse_columns(ranking, header)

        # find positions for each field
        start = 0
        cols = {} 
        unnamed = 0
        for delimiter in delimiters:
            key = header[start:delimiter+1].strip()
            if key == '':
                key = unnamed
                unnamed += 1
            cols[key] = (start, delimiter+1)
            start = delimiter+1

        start = 0
        result = {}
        for i, line in enumerate(ranking):
            for col in cols:
                start, end = cols[col]
                if col in result:
                    result[col].append(line[start:end].strip())
                else:
                    result[col] = [line[start:end].strip()]
        dfs[year] = pd.DataFrame(result, columns=result.keys()) 
    
    return dfs

dfs = separate_cols(rankings, headers)

Skipped line:
 
total 353

(13,) (170,)
Skipped line:
 
total 353

(13,) (170,)
Skipped line:
 
total 353

(13,) (170,)
Skipped line:
 
total 353

(13,) (170,)
Skipped line:
 
total 353

(13,) (170,)
Skipped line:
 
total 353

(13,) (170,)
Skipped line:
 
total 353

(13,) (170,)
Skipped line:
 
total 353

(13,) (170,)
Skipped line:
 
total 353

(13,) (170,)
Skipped line:
 
total 353

(13,) (170,)
Skipped line:
 
total 353

(13,) (170,)
Skipped line:
 
total 353

(13,) (170,)
Skipped line:
 
total 353

(13,) (170,)
Skipped line:
 
total 353

(13,) (170,)
Skipped line:
 
total 353

(13,) (170,)
Skipped line:
 
total 353

(13,) (170,)
Skipped line:
 
total 353

(13,) (170,)
Skipped line:
 
total 353

(13,) (170,)
Skipped line:
 
total 353

(13,) (170,)
Skipped line:
 
total 353

(13,) (170,)
Skipped line:
 
total 353

(13,) (170,)
Skipped line:
 
total 353

(13,) (170,)
Skipped line:
 
total 353

(13,) (170,)
Skipped line:
 
total 353

(13,) (170,)
Skipped line:
 
total 353

(13,) (170,)


In [54]:
common_fields = set()
for year in dfs:
    print(year)
    print(list(dfs[year]))
    print(dfs[year].shape)


2011
['lieu', 0, 2, 'catégorie', 'an', 'nom', 'retard', 'temps', 'doss', 'équipe', 'pénalité', 1, 'moyenne', 'rang']
(9204, 14)
2005
['rang', 'équipe/lieu', 'catégorie', 'nom', 'an', 'retard', 'temps', 'doss']
(7748, 8)
2016
['lieu', 0, 2, 3, 'catégorie', 'an', 'nom', 'retard', 'temps', 'doss', 'équipe', 'pénalité', 1, 'moyenne', 'rang', 'overall']
(13130, 16)
2013
['lieu', 1, 2, 'catégorie', 'an', 'nom', 'rang', 'temps', 'doss', 'équipe', 'retard', 'moyenne', 0]
(11362, 13)
2009
[0, 'rang', 'catégorie', 'nom', 'an lieu', 'temps', 'retard', 'doss', '¦']
(8190, 9)
2015
['lieu', 0, 2, 3, 'catégorie', 'an', 'nom', 'retard', 'temps', 'doss', 'équipe', 'pénalité', 1, 'moyenne', 'rang', 'overall']
(12012, 16)
2002
['Rang', 'Stnr', 'Rückstand', 'Name/Ort', 'Team/Ortschaft', 'Jg', 'Kategorie', 'Zeit']
(5018, 8)
2012
['lieu', 1, 2, 'catégorie', 'an', 'nom', 'rang', 'temps', 'doss', 'équipe', 'retard', 'moyenne', 0]
(8918, 13)
2004
['rang', 'équipe/lieu', 'catégorie', 'an', 'nom/lieu', 'temps', 